In [ ]:
# pip install -qU langchain qdrant-client openai pypdf pypdf2 langchain-openai langchain-community uuid tiktoken pikepdf sentence_transformers langchain-groq

In [11]:
import os
import time
import asyncio
from langchain_openai import OpenAIEmbeddings

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant

from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_community.vectorstores import Qdrant
from langchain.schema import retriever

In [13]:
# Load the environment variables
qdrant_host = os.getenv('QDRANT_HOST')
# openai_api_key = os.getenv('OPENAI_API_KEY')

In [14]:
print(qdrant_host)

https://qdrant-dev.yirifi.ai:443


In [1]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Initialize qdrant client
client = QdrantClient(url=qdrant_host)

In [16]:
# Get list of collections present in qdrant store
collections = client.get_collections()

In [17]:
# Print list of collections
list_of_collections = collections.collections
for collection in list_of_collections:
  print(collection.name)

llamaparse pdf file
Empathy_Tales
RegDb vectors 1000-chars-sb
llamaparse pl 256-tokens
RegDb Vectors v1
RegDb Vectors v2
test 768-dim
test
RegDb Vectors
llamaparse 3-pdf files
New-768-dim-sb


In [18]:
# Connect to a document store
doc_store = Qdrant(
        client=client,
        collection_name='Empathy_Tales', # Can change the collection here
        embeddings = hf
        )

In [ ]:
# initialize openai llm
# llm = OpenAI(api_key=openai_api_key)

In [19]:
from langchain_groq import ChatGroq

In [20]:
import os
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768", groq_api_key='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')

In [21]:
qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type="stuff",
        retriever= doc_store.as_retriever(),
        return_source_documents=True
    )

In [23]:
user_query = 'What is the moral of the story of water taking any shape of a container?'
response = qa.invoke(user_query)

In [26]:
print(f"User Query: {response['query']}")
print(f"Answer: {response['result']}")

User Query: What is the moral of the story of water taking any shape of a container?
Answer: The moral of the story of water taking any shape of a container is: "The importance of considering multiple perspectives and conditions before making assumptions or conclusions." This story illustrates that there are often factors (like temperature, in this case) that can significantly impact the outcome of a situation, and it's crucial to take these factors into account before drawing conclusions.
